# Set-Up

In [128]:
import numpy as np
import codecs

In [ ]:
hex_dict={'0':0,'1':1,'2':2,'3':3,'4':4,'5':5,'6':6,'7':7,'8':8,'9':9,'a':10,'b':11,'c':12,'d':13,'e':14,'f':15}
S=[0,2,10,9,6,4,14,13,1,7,15,8,11,12,3,5]
p=[0,5,10,15,4,9,14,3,8,13,2,7,12,1,6,11]

In [ ]:
#Print State Funciton
def print_s(state,message):
  if debug==True:
    print(message)
    print()
    for i in range(4):
        for j in range(4):
          t=str(hex(state[i][j]))[2:]
          if len(t)<2:
            t='0'+t
          print(t,' ',end='')
        print()
    print()
  else:
    return None

# Functions for encryption 

In [ ]:
#S-box

def Sbox(n):
  a=str(hex(n))[2:]
  if len(a)<2:
          a='0'+a
  a1=hex_dict[a[0]]
  a2=hex_dict[a[1]]
  r=S[a1]*16+S[a2]
  return r


In [ ]:
#L-box

def lbox(state):
    state_8=[[],[],[],[]]
    for i in range(4):
      for j in range(4):
        a=str(hex(state[i][j]))[2:]
        if len(a)<2:
          a='0'+a
        a1=hex_dict[a[0]]
        a2=hex_dict[a[1]]
        state_8[i].append(a1)
        state_8[i].append(a2)
    poly = [0, 0b1000, 0b0011, 0b1111, 0b0101, 0b1111, 0b0011, 0b1000]
    value=[]
    for c in range(4):
      for _ in range(8):
          x = state_8[c][0]
          for i in range(8):
              x ^= MultiplyGF16(state_8[c][i], poly[i])
          state_8[c].pop(0)
          state_8[c].append(x)
    for e in range(4):
      for f in range(4):
        state[e][f]=16*state_8[e][2*f]+state_8[e][2*f+1]
    return state

def MultiplyGF16(a, b, p=0b10011):
    result = 0
    for _ in range(4):
        result ^= (b & 1) * a
        a <<= 1
        a ^= (a >> 4) * p
        b >>= 1
    return result


In [ ]:
#Shift Column

def ShiftColumns_128(state):
  lstate=[]
  for i in range(4):
    for j in range(4):
      lstate.append(state[i][j])
  plstate=[0]*16
  for i in range(16):
    plstate[p[i]]=lstate[i]
  value=[]
  for i in range(4):
    value.append(plstate[4*i:4*i+4])
  return value

In [ ]:
# Round Constants

def roundconst(i):
    r=[[[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,1]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,2]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,4]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,8]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,16]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,32]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,64]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,128]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,27]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,54]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,108]],
       [[0,0,0,0],[0,0,0,0],[0,0,0,0],[0,0,0,40]]]
    return r[i]

In [108]:
def Mysterion128(key, m, NR):
  temp=''
  # adding key initially
  print_s(m,'Initially Message is:')
  print() 

  state = np.array( [x ^ y for x, y in zip(key, m)] )
  #print(state)
  if (debug or rounds):
    print('After Adding K_0:')
    print()
    for i in range(4):
      for j in range(4):
        t=str(hex(state[i][j]))[2:]
        if len(t)<2:
          t='0'+t
        print(t,' ',end='')
      print() 

  for round in range(1, NR + 1):
    if debug:
      print('------------Round No'+' : '+str(round)+'------------------')
      print()
    # S-box
    for i in range(4):
      for j in range(4):
        tp=Sbox(state[i][j])
        state[i][j]=tp
    print_s(state,'After S-box Operation:')

    # L-box
    for i in range(4):
      for j in range(4):
        state=lbox(state)
    print_s(state,'After L-box Operation:')

    # ShiftColumns 128-bit
    state = ShiftColumns_128(state)
    print_s(state,'After Shift Column:')

    #key addition
    state = np.array( [x ^ y for x, y in zip(key, state)])
    print_s(state,'After Key Addition:')

    #round Constant
    rc=roundconst(round-1)
    state = np.array( [x ^ c for x, c in zip(state, rc)])
    print_s(state,'After Round Constant:')

    if rounds==True:
      print()
      print('Round No : '+str(round))
      for i in range(4):
        for j in range(4):
          t=str(hex(state[i][j]))[2:]
          if len(t)<2:
            t='0'+t
          print(t,' ',end='')
        print()
      print()
  
  print('Ciphertext is: ')
  for i in range(4):
      for j in range(4):
        t=str(hex(state[i][j]))[2:]
        if len(t)<2:
          t='0'+t
        temp+=t
  temp=temp.upper()
  print(temp)
  return temp

# Functions For Decryption

In [157]:
def sbox_rev(n):
    a=str(hex(n))[2:]
    if len(a)<2:
            a='0'+a
    a1=hex_dict[a[0]]
    a2=hex_dict[a[1]]
    
    r=S.index(a1)*16+S.index(a2)
    return r

In [156]:
def rev_lbox(state):
    state_8=[[],[],[],[]]
    for i in range(4):
      for j in range(4):
        a=str(hex(state[i][j]))[2:]
        if len(a)<2:
          a='0'+a
        a1=hex_dict[a[0]]
        a2=hex_dict[a[1]]
        state_8[i].append(a1)
        state_8[i].append(a2)
    
    poly = [0b1000, 0b0011, 0b1111, 0b0101, 0b1111, 0b0011, 0b1000, 0]
    value=[]
    for c in range(4):
      for _ in range(8):
          x = state_8[c][7]
         
          for i in range(8):
              x ^= MultiplyGF16(state_8[c][i], poly[i])
              
          state_8[c].pop(7)
          state_8[c]=[x]+state_8[c][:]

    for e in range(4):
      for f in range(4):
        state[e][f]=16*state_8[e][2*f]+state_8[e][2*f+1]

    return state

In [ ]:
def ShiftColumns_128_inv(state):
    lstate=[]
    for i in range(4):
      for j in range(4):
        lstate.append(state[i][j])
    plstate=[0]*16
    for i in range(16):
      plstate[i]=lstate[p[i]]
    value=[]
    for i in range(4):
      value.append(plstate[4*i:4*i+4])
    return value

In [111]:
def InvMysterion128(c, key, NR):
  state=c
  for round in range(NR, 0, -1):
    if rounds==True:
      print()
      print('Round No : '+str(round))
      for i in range(4):
        for j in range(4):
          t=str(hex(state[i][j]))[2:]
          if len(t)<2:
            t='0'+t
          print(t,' ',end='')
        print()
      print()

    if debug:
      print('----------Round No'+' : '+str(round)+'----------')
      print_s(state,'')


    rc=roundconst(round-1)

    for h in range(4):
      for g in range(4):
        f=state[h][g]
        state[h][g]=f^rc[h][g]
    print_s(state,'After Rev-Constant:')

    #revrse key
    state = np.array( [a^b for a, b in zip(state, key)])
    print_s(state,'After Rev-Key:')
    
    #revrse shift-row
    state = ShiftColumns_128_inv(state)
    print_s(state,'After Rev-ShiftRow:')

    # L-box inverse
    for i in range(4):
      for j in range(4):
        state=rev_lbox(state)
    print_s(state,'After Rev-L-box:')

    # S-box inverse
    for i in range(4):
      for j in range(4):
        tp=sbox_rev(state[i][j])
        state[i][j]=tp
    print_s(state,'after Rev-sbox')


  #intial key removal
  state = np.array( [x ^ y for x, y in zip(state, key)] )
  if (debug or rounds):
    print('After Removing K_0:')
    print()
    for i in range(4):
      for j in range(4):
        t=str(hex(state[i][j]))[2:]
        if len(t)<2:
          t='0'+t
        print(t,' ',end='')
      print() 
  
  print('Message is:')
  temp=''
  for i in range(4):
      for j in range(4):
        t=str(hex(state[i][j]))[2:]
        if len(t)<2:
          t='0'+t
        temp+=t
  temp=temp.upper()
  print(temp)
  return temp

# Encryption and Decryption

In [ ]:
def encrypt(m, k, r):

    m = list(bytearray.fromhex(m))
    k = list(bytearray.fromhex(k))

    m = np.array([m[i*4:(i+1)*4] for i in range(4)])
    k = np.array([k[i*4:(i+1)*4] for i in range(4)])

    ct = Mysterion128(k, m, r)
    
    return ct

In [124]:
def decrypt(c, k, r):
    c = list(bytearray.fromhex(c))
    k = list(bytearray.fromhex(k))

    # spliting message in blocks 
    c = np.array([c[i*4:(i+1)*4] for i in range(4)])
    k = np.array([k[i*4:(i+1)*4] for i in range(4)])

    cpt = InvMysterion128(c, k, r)
    return cpt

# Running

In [158]:
inp='SHA 69 is best'

#String To Hex
message="".join(["{:02x}".format(ord(c)) for c in inp])
#Padding
if len(message)<32:
  message=message+'0'*(32-len(message))
if len(message)>32:
  message=message[:32]

message=message.upper()

key='0205060752F3E1F2132435465B6C7D88'
r=12
debug=False
rounds=True
ciphertext= encrypt(message,key,r)



After Adding K_0:

51  4d  47  27  
64  ca  c1  9b  
60  04  57  23  
28  18  7d  88  

Round No : 1
00  16  d0  11  
a9  12  f3  fd  
12  82  12  eb  
05  f1  a3  f6  


Round No : 2
0b  41  84  3a  
85  28  9d  34  
cf  77  17  24  
05  71  b7  fd  


Round No : 3
ed  e5  1a  04  
76  52  14  91  
e9  be  dd  50  
2b  2b  0e  17  


Round No : 4
a0  d5  5f  b1  
d2  9b  30  a7  
81  b2  96  6b  
55  6a  95  40  


Round No : 5
6f  87  9f  57  
9a  00  0c  d2  
a2  33  e0  6d  
19  b2  4b  4e  


Round No : 6
b5  16  d0  06  
c4  fc  c0  6c  
48  ac  68  e0  
6c  d3  48  32  


Round No : 7
dd  b2  74  cc  
51  ad  2f  52  
84  74  5b  8b  
00  46  b8  a0  


Round No : 8
a8  c9  d2  2b  
16  d4  76  e8  
e7  17  ff  62  
51  f8  1d  31  


Round No : 9
18  e0  43  7c  
45  5c  d6  96  
8c  63  cd  c9  
f0  ee  6e  b8  


Round No : 10
da  02  05  9c  
3e  68  8f  7e  
08  8e  e6  d8  
34  20  07  39  


Round No : 11
5b  21  b3  67  
3a  c7  fe  09  
d2  ae  f6  93  
c7  57  d2  b2 

In [159]:
d=decrypt(ciphertext,key,r)

d_a = codecs.decode(d, "hex")
print('Ascii is: ',str(d_a,'utf-8'))

if len(inp)<16:
  d=d[:len(d)-2*(16-len(inp))]
  d_a = codecs.decode(d, "hex")
  print()
  print('Ascii(without garbage) is: ')
  print(str(d_a,'utf-8'))


Round No : 12
d6  b7  23  2a  
3f  1a  51  93  
d9  c1  47  33  
17  c6  08  a6  


Round No : 11
5b  21  b3  67  
3a  c7  fe  09  
d2  ae  f6  93  
c7  57  d2  b2  


Round No : 10
da  02  05  9c  
3e  68  8f  7e  
08  8e  e6  d8  
34  20  07  39  


Round No : 9
18  e0  43  7c  
45  5c  d6  96  
8c  63  cd  c9  
f0  ee  6e  b8  


Round No : 8
a8  c9  d2  2b  
16  d4  76  e8  
e7  17  ff  62  
51  f8  1d  31  


Round No : 7
dd  b2  74  cc  
51  ad  2f  52  
84  74  5b  8b  
00  46  b8  a0  


Round No : 6
b5  16  d0  06  
c4  fc  c0  6c  
48  ac  68  e0  
6c  d3  48  32  


Round No : 5
6f  87  9f  57  
9a  00  0c  d2  
a2  33  e0  6d  
19  b2  4b  4e  


Round No : 4
a0  d5  5f  b1  
d2  9b  30  a7  
81  b2  96  6b  
55  6a  95  40  


Round No : 3
ed  e5  1a  04  
76  52  14  91  
e9  be  dd  50  
2b  2b  0e  17  


Round No : 2
0b  41  84  3a  
85  28  9d  34  
cf  77  17  24  
05  71  b7  fd  


Round No : 1
00  16  d0  11  
a9  12  f3  fd  
12  82  12  eb  
05  f1  a3  f6  

A